In [1]:
import pandas as pd
import requests
from io import StringIO
import numpy as np
import uuid

Steps: we import the basart_address from OR and then merge it with the v_address and the address table from basart db. The v_address does not have city_id, but it has coordinates. The address does not have coordinates but it has city_id. The basart_address from OR has addresses that have been cleaned and confirmed (but only paris)

In [2]:
url = "http://129.194.213.75/command/core/export-rows"
params = {
    'format': 'csv',
    'project': '1969739192362'
}

response = requests.post(url, params=params)

if response.status_code == 200:
    df = pd.read_csv(StringIO(response.text), dtype ='str', low_memory=False)
else:
    print(f"Failed to load data. Status code: {response.status_code}")

In [3]:
add_basart = pd.read_csv("/Users/carboni/Downloads/v_address leboulanger.csv",  dtype ='str')

In [4]:
add_basart_city = pd.read_csv("/Users/carboni/Downloads/Address.csv",  dtype ='str')

In [5]:
add_basart_city_id = add_basart_city[['id','id_city']]

In [6]:
merged_bas = add_basart.merge(add_basart_city_id, on='id', how='left')

In [7]:
merged_bas = merged_bas.drop(columns=['city_x', 'city_y', 'statefr', 'stateen', 'countryen', 'countryfr'])

we eliminate nan from x/y in merged bas

In [8]:
merged_bas.dropna(subset=['x'], inplace=True)

In [9]:
merged_bas['address_combined'] = merged_bas.apply(lambda row: ' '.join(str(row[col]) for col in ['road', 'road_num', 'floor', 'locality', 'complement'] if pd.notna(row[col])), axis=1)

In [10]:
merged_bas.head()

,id,road_num,road,floor,locality,complement,x,y,city,id_city,address_combined
6,3898,3,rue de la Promenade,NaN,NaN,NaN,2.286761,48.912498,Asnières,2290,rue de la Promenade 3
7,3900,13,rue Eugénie-Gérard,NaN,NaN,NaN,2.4330176,48.8462203,Vincennes,2293,rue Eugénie-Gérard 13
12,3907,7 bis,rue de la Villa,NaN,NaN,NaN,2.2748868,48.8522421,Sèvres,2303,rue de la Villa 7 bis
21,3902,23,avenue du Centenaire,NaN,NaN,NaN,2.5076739,48.7976899,La Varenne-Saint-Hilaire,96763,avenue du Centenaire 23
23,3938,64,boulevard de l'Ouest,NaN,NaN,NaN,2.5095775,48.897598,Le Raincy,97228,boulevard de l'Ouest 64


we keep only the curated values (all_lat/all_long) from df

In [11]:
df.dropna(subset=['all_lat'], inplace=True)

In [12]:
df_curated = df[['id','all_long', 'all_lat']]

In [13]:
df_curated.head(5)

,id,all_long,all_lat
16,3878,2.3449549,48.8534925
29,22104,2.37552,48.8393898
31,22105,2.3721447,48.8612561
32,22106,2.2755728,48.8594591
33,22107,2.3481673,48.8420621


In [14]:
merged = pd.merge(merged_bas, df_curated[['id', 'all_lat', 'all_long']], on='id', how='left')

In [15]:
# Update 'y' and 'x' in merged DataFrame with values from 'all_lat' and 'all_long' if available
merged['y'] = merged['all_lat'].combine_first(merged['y'])
merged['x'] = merged['all_long'].combine_first(merged['x'])

# Drop the temporary columns 'all_lat' and 'all_long'
merged.drop(columns=['all_lat', 'all_long'], inplace=True)

In [16]:
merged.rename(columns={'x': 'lng', 'y': 'lat'}, inplace=True)

In [22]:
merged["wkt"] = "POINT(" + merged["lng"] + " " + merged["lat"] + ")"

In [23]:
merged.head(5)

,id,road_num,road,floor,locality,complement,lng,lat,city,id_city,address_combined,wkt
0,3898,3,rue de la Promenade,NaN,NaN,NaN,2.286761,48.912498,Asnières,2290,rue de la Promenade 3,POINT(2.286761 48.912498)
1,3900,13,rue Eugénie-Gérard,NaN,NaN,NaN,2.4330176,48.8462203,Vincennes,2293,rue Eugénie-Gérard 13,POINT(2.4330176 48.8462203)
2,3907,7 bis,rue de la Villa,NaN,NaN,NaN,2.2748868,48.8522421,Sèvres,2303,rue de la Villa 7 bis,POINT(2.2748868 48.8522421)
3,3902,23,avenue du Centenaire,NaN,NaN,NaN,2.5076739,48.7976899,La Varenne-Saint-Hilaire,96763,avenue du Centenaire 23,POINT(2.5076739 48.7976899)
4,3938,64,boulevard de l'Ouest,NaN,NaN,NaN,2.5095775,48.897598,Le Raincy,97228,boulevard de l'Ouest 64,POINT(2.5095775 48.897598)


In [18]:
merged.to_csv('/Users/carboni/Downloads/merged.csv', index=False)

In [24]:
columns = merged.columns

# Iterate through the columns
for col in columns:
    # Check if the column name contains a space
    if ' ' in col:
        # Replace spaces with underscores
        merged.rename(columns={col: col.replace(' ', '_')}, inplace=True)

In [25]:
merged = merged.replace(r"^ +| +$", r"", regex=True)

In [26]:
merged.to_xml('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/address/address.xml', root_name="data", pretty_print=True, index=False) 